In this notebook, we will build a 3D map of a scene from a small set of images and then localize an image downloaded from the Internet. This demo was contributed by [Philipp Lindenberger](https://github.com/Phil26AT/).

In [ ]:
!pip install pycolmap==3.10.0
!pip install kornia

In [ ]:
import os
from pathlib import Path
import pycolmap
print(pycolmap.__version__)
!pwd

In [ ]:
# For experiments in Colab (for gpu)
!git clone --quiet --recursive https://github.com/ikotecha/18794-Project

from Project.DeepUWSFM.utils import viz, viz_3d
from Project.DeepUWSFM.utils.io import read_image
from Project.DeepUWSFM import (
    feature_extractor, dense_matcher, feature_matcher, pairing, triangulation, reconstruction
)

In [40]:
import tqdm, tqdm.notebook
tqdm.tqdm = tqdm.notebook.tqdm

from DeepUWSFM.utils import viz, viz_3d
from DeepUWSFM import (
    feature_extractor, dense_matcher, feature_matcher, pairing, pycolmap_db, reconstruct, config
)

In [43]:
images = Path("images")

outputs = Path("outputs")
sfm_pairs = outputs / "pairs.txt"
sfm_dir = outputs / "sfm_superpoint+superglue"
sfm_dir = outputs / 'sfm'
features = outputs / 'features.h5'
matches = outputs / 'matches.h5'

feature_conf = config.configs["superpoint"]
dense_conf = config.configs['loftr']
matcher_conf = config.configs["superglue"]

In [ ]:
references = [str(p.relative_to(images)) for p in (images).iterdir()]
print(len(references), "mapping images")
viz.plot_images([read_image(images / r) for r in references], dpi=25)

feature_extractor.extract_features(feature_conf, images, img_list=references, feat_path=features)
pairing.main(sfm_pairs, imgs=references)
feature_matcher.match_from_paths(matcher_conf, sfm_pairs, feature_path_q=features, match_path=matches)

features_loftr, matches_loftr = dense_matcher.main(dense_conf, sfm_pairs, images, export_dir=outputs)

In [ ]:
model = reconstruct.main(sfm_dir, images, sfm_pairs, features_loftr, matches_loftr, image_list=references)
print(model.summary())

In [ ]:
import pycolmap

def save_points3D_to_ply(reconstruction, output_file):
    points3D = reconstruction.points3D
    with open(output_file, "w") as f:
        f.write("ply\n")
        f.write("format ascii 1.0\n")
        f.write(f"element vertex {len(points3D)}\n")
        f.write("property float x\n")
        f.write("property float y\n")
        f.write("property float z\n")
        f.write("property uchar red\n")
        f.write("property uchar green\n")
        f.write("property uchar blue\n")
        f.write("end_header\n")

        for point_id, point in points3D.items():
            x, y, z = point.xyz
            r, g, b = point.color
            f.write(f"{x} {y} {z} {r} {g} {b}\n")

    print(f"Point cloud saved to {output_file}")

output_ply_file = images / "points3D.ply"
save_points3D_to_ply(rec, output_ply_file)